# Descriptive Statistics
### base on Chua, Alton Y.K. and Banerjee, Snehasish 2016
#### http://www.sciencedirect.com/science/article/pii/S074756321530131X

In [1]:
import sys
sys.path.append("../pylinguistics/pylinguistics/")

#import Pylinguistics as pl
import pandas as pd
import plotsfunc as pf
import numpy as np
%matplotlib inline

#reviews_features = pd.read_csv('reviews.csv.gz', compression='gzip')
reviews_features = pd.read_csv('amazon-help.csv.gz')

In [2]:
## adiciona coluna de helpfulness no corpus
def helpf(x): 
    try:
        return float ( float(x['thumbsup']) / (float(x['thumbsup']) + float(x['thumbsdown'])) )
    except:
        return 0
    
def thumbssum(x): 
    try:
        return int(x['thumbsup']) + int(x['thumbsdown']) 
    except:
        return 0
    
def fillnanup(x): 
    try:
        return float(x['thumbsup'])
    except:
        return 0
    
def fillnandown(x): 
    try:
        return float(x['thumbsdown'])
    except:
        return 0
    
def length(x): 
    try:
        return int(x['word_count']) * int(x['mean_word_length'])
    except:
        return 0
    
def sentiment(x): 
    if x['stars'] == 5:
        return 'favorable'
    if x['stars'] == 1:
        return 'unfavorable'
    else:
        return 'mixed'

In [3]:
reviews_features['thumbsup'] = reviews_features.apply(fillnanup,axis=1)
reviews_features['thumbsdown'] = reviews_features.apply(fillnandown,axis=1)
reviews_features['helpfulness'] = reviews_features.apply(helpf,axis=1)
reviews_features['thumbstotal'] = reviews_features.apply(thumbssum,axis=1)
reviews_features['length'] = reviews_features.apply(length,axis=1)
reviews_features = reviews_features[reviews_features.thumbstotal>4]
reviews_features = reviews_features[reviews_features.word_count>9]
#reviews_features = reviews_features[~reviews_features.applymap(np.isnan).all(1)]
## para usar no weka
#reviews_features['sentiment'] = reviews_features.apply(sentiment,axis=1)
#reviews_features.to_csv('buscape-help.csv',encoding='utf-8')
reviews_features.shape

(15104, 47)

In [4]:
def mean_data(df):
    mean_stats = []
    mean_stats.append(df.thumbstotal.mean())
    mean_stats.append(df.thumbsup.mean())
    mean_stats.append(df.thumbsdown.mean())
    mean_stats.append(df.stars.mean())
    mean_stats.append(df.word_count.mean())
    mean_stats.append(df.helpfulness.mean())
    mean_stats.append(len(df))
    return mean_stats

def std_data(df):
    std_stats = []
    std_stats.append(df.thumbstotal.std())
    std_stats.append(df.thumbsup.std())
    std_stats.append(df.thumbsdown.std())
    std_stats.append(df.stars.std())
    std_stats.append(df.word_count.std())
    std_stats.append(df.helpfulness.std())
    std_stats.append(len(df))
    return std_stats

columns = ['TOT','UP','DOWN','RAT','DEP','HEL','LEN']

In [5]:
# split mixed subset
mixed_reviews = reviews_features[reviews_features.stars > 1]
mixed_reviews = mixed_reviews[reviews_features.stars < 5]

/home/henrique/.local/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [6]:
# split favorable subset
favorable_reviews = reviews_features[reviews_features.stars == 5]

In [7]:
# split unfavorable subset
unfavorable_reviews = reviews_features[reviews_features.stars == 1]

In [8]:
stats = pd.DataFrame(data={'mean_favorable': np.array(mean_data(favorable_reviews))}, index=columns)
stats['std_favorable'] = std_data(favorable_reviews)
stats['mean_unfavorable'] = mean_data(unfavorable_reviews)
stats['std_unfavorable'] = std_data(unfavorable_reviews)
stats['mean_mixed'] = mean_data(mixed_reviews)
stats['std_mixed'] = std_data(mixed_reviews)

## Descriptive Table Amazon Reviews
### statistics (mean ± sd) of reviews as a function of review sentiment

In [9]:
stats

,mean_favorable,std_favorable,mean_unfavorable,std_unfavorable,mean_mixed,std_mixed
TOT,20.357340,47.932518,17.225176,36.197885,19.846433,52.053078
UP,9.510823,23.615432,6.882257,17.323194,8.985953,25.521014
DOWN,10.846517,24.419609,10.342919,19.371485,10.860480,26.712077
RAT,5.000000,0.000000,1.000000,0.000000,3.228851,0.825613
DEP,161.513971,206.319770,117.622355,129.551480,182.135417,193.377809
HEL,0.452487,0.088056,0.386303,0.129808,0.435749,0.100368
LEN,5082.000000,5082.000000,3686.000000,3686.000000,6336.000000,6336.000000


## Results of the multiple regression analyses

In [10]:
from scipy.stats import spearmanr, f_oneway
from sklearn.svm import SVR, LinearSVR
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import make_scorer

def simple_spearman(x,y): return np.abs(spearmanr(x,y)[0])
spearmanr_scorer = make_scorer(simple_spearman)

In [11]:
# create new dict for correlation
mixed_list = []
mixed_list_p = []
favorable_list = []
favorable_list_p = []
unfavorable_list = []
unfavorable_list_p = []
columns_pd = []

#svm model
model = LinearSVR(epsilon=0.01)

# build columns list
df_columns = mixed_reviews.select_dtypes(include=['float64','int']).columns.drop('helpfulness')
df_columns

Index([u'ConnectiveAdditiveIncidence', u'redability', u'mean_sentence_length',
       u'percentile_75_sentence_length', u'percentile_75_word_length',
       u'ConnectiveLogicIncidence', u'percentile_90_word_length',
       u'mean_word_length', u'syllable_count', u'avg_word_per_sentence',
       u'LogicIfIncidence', u'percentile_50_sentence_length',
       u'median_sentence_length', u'LogicAndIncidence',
       u'percentile_90_sentence_length', u'median_word_length',
       u'pronIncidence', u'LogicOperatorsIncidence',
       u'percentile_25_word_length', u'ContentDiversty', u'verbIncidence',
       u'functionalIncidence', u'nounIncidence', u'percentile_50_word_length',
       u'percentile_25_sentence_length', u'LogicOrIncidence',
       u'adjectiveIncidence', u'ConnectiveIncidence', u'contentIncidence',
       u'LogicNegationIncidence', u'avg_syllables_per_word',
       u'ConnectiveTemporalIncidence', u'ConnectiveCasualIncidence',
       u'advIncidence', u'LexicalDiversty', u'adpPronRa

In [12]:
#compute correlation
for i in df_columns:
    #compute mixed score
    labels = mixed_reviews["helpfulness"].values
    features = mixed_reviews[list([i])].values
    features_flat = mixed_reviews[i].values
    scores = cross_val_score(model, features, labels, cv=5, scoring=spearmanr_scorer)
    f_val,p_val = f_oneway(features_flat, labels)
    mixed_list_p.append(p_val)
    mixed_list.append(scores.mean())
    #compute favorable score
    labels = favorable_reviews["helpfulness"].values
    features = favorable_reviews[list([i])].values
    features_flat = favorable_reviews[i].values
    scores = cross_val_score(model, features, labels, cv=5, scoring=spearmanr_scorer)
    f_val,p_val = f_oneway(features_flat, labels)
    favorable_list_p.append(p_val)
    favorable_list.append(scores.mean())
    #compute unfavorable score
    labels = unfavorable_reviews["helpfulness"].values
    features = unfavorable_reviews[list([i])].values
    features_flat = unfavorable_reviews[i].values
    scores = cross_val_score(model, features, labels, cv=5, scoring=spearmanr_scorer)
    f_val,p_val = f_oneway(features_flat, labels)
    unfavorable_list_p.append(p_val)
    unfavorable_list.append(scores.mean())
    #print status
    columns_pd.append(i);
    print i
    #if len(columns_pd) > 3:
    #    break

ConnectiveAdditiveIncidence
redability
mean_sentence_length
percentile_75_sentence_length
percentile_75_word_length
ConnectiveLogicIncidence
percentile_90_word_length
mean_word_length
syllable_count
avg_word_per_sentence
LogicIfIncidence
percentile_50_sentence_length
median_sentence_length
LogicAndIncidence
percentile_90_sentence_length
median_word_length
pronIncidence
LogicOperatorsIncidence
percentile_25_word_length
ContentDiversty
verbIncidence
functionalIncidence
nounIncidence
percentile_50_word_length
percentile_25_sentence_length
LogicOrIncidence
adjectiveIncidence
ConnectiveIncidence
contentIncidence
LogicNegationIncidence
avg_syllables_per_word
ConnectiveTemporalIncidence
ConnectiveCasualIncidence
advIncidence
LexicalDiversty
adpPronRatio
thumbsup
thumbsdown
stars


In [13]:
corr_df = pd.DataFrame(data={'favorable': np.array(favorable_list)}, index=columns_pd)
corr_df['fav_p'] = favorable_list_p
corr_df['unfavorable'] = unfavorable_list
corr_df['unfav_p'] = unfavorable_list_p
corr_df['mixed'] = mixed_list
corr_df['mix_p'] = mixed_list_p

## Amazon Correlation Features

In [14]:
corr_df.sort_values('favorable',ascending=False)

,favorable,fav_p,unfavorable,unfav_p,mixed,mix_p
thumbsup,0.259014,6.487837e-159,0.478415,6.270130e-111,0.351214,1.736382e-152
syllable_count,0.218747,0.000000e+00,0.048230,0.000000e+00,0.201351,0.000000e+00
LexicalDiversty,0.180127,0.000000e+00,0.038120,0.000000e+00,0.171709,0.000000e+00
thumbsdown,0.164538,1.134290e-193,0.247389,6.943294e-201,0.157841,2.995301e-204
percentile_90_sentence_length,0.129964,0.000000e+00,0.019393,0.000000e+00,0.108433,0.000000e+00
ContentDiversty,0.117680,0.000000e+00,0.033474,0.000000e+00,0.111954,0.000000e+00
contentIncidence,0.115403,0.000000e+00,0.045382,0.000000e+00,0.091873,0.000000e+00
LogicOrIncidence,0.109832,2.307627e-171,0.028846,3.844650e-119,0.072755,1.159052e-280
percentile_75_sentence_length,0.106541,0.000000e+00,0.020318,0.000000e+00,0.082970,0.000000e+00
percentile_25_word_length,0.105512,0.000000e+00,0.061645,0.000000e+00,0.064378,0.000000e+00
